In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Groupby and Aggregrate Operations").getOrCreate()

In [5]:
df=spark.read.csv('C:/Users/User/Desktop/SparkFolder/Data/sales_info.csv',inferSchema=True, header=True)

In [8]:
df.show(2)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+
only showing top 2 rows



In [9]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



# Group by 

In [10]:
df.groupby('Company')

In [11]:
df.groupby('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [21]:
# Mostly Aggregrate functions you know will work
df.groupby('Company').sum().collect()[0].asDict()

{'Company': 'APPL', 'sum(Sales)': 1480.0}

In [22]:
df.groupby('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [24]:
df.groupby('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [26]:
df.groupby('Company').max().withColumnRenamed("max(Sales)","sales").show()

+-------+-----+
|Company|sales|
+-------+-----+
|   APPL|750.0|
|   GOOG|340.0|
|     FB|870.0|
|   MSFT|600.0|
+-------+-----+



In [29]:
df.select("Sales").show()

+-----+
|Sales|
+-----+
|200.0|
|120.0|
|340.0|
|600.0|
|124.0|
|243.0|
|870.0|
|350.0|
|250.0|
|130.0|
|750.0|
|350.0|
+-----+



In [33]:
df.groupby('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [43]:
df.groupby('Company').avg().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



# .Agg

In [37]:
df.agg({"Sales":'avg'}).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [36]:
df.agg({"Sales":'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



# Import Functions

In [39]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [42]:
df.select(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [44]:
df.select(stddev("Sales")).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [45]:
df.select(avg("Sales")).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



# .Alias 
* Used to rename Columns

In [48]:
df.agg({"Sales":'sum'}).alias ("Sum of Sales").show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [49]:
df.select(countDistinct("Sales").alias("Distinct Count")).show()

+--------------+
|Distinct Count|
+--------------+
|            11|
+--------------+



In [51]:
df.select(stddev("Sales").alias('STD')).show()

+------------------+
|               STD|
+------------------+
|250.08742410799007|
+------------------+



# .Format_Number

In [52]:
from pyspark.sql.functions import format_number

In [61]:
std=df.select(stddev("Sales").alias('STD'))

In [62]:
std.show()

+------------------+
|               STD|
+------------------+
|250.08742410799007|
+------------------+



In [63]:
std.select("STD").show()

+------------------+
|               STD|
+------------------+
|250.08742410799007|
+------------------+



In [65]:
std.select(format_number("STD",2).alias("STD")).show()

+------+
|   STD|
+------+
|250.09|
+------+



# Order and Sort 

In [67]:
df.orderBy("Sales").show()  #ascending

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [70]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [72]:
# Doing this in SQL
df.createOrReplaceTempView("Software")

In [81]:
results=spark.sql("Select * from Software \
                  order by Sales asc")

In [82]:
results.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [83]:
results=spark.sql("Select * from Software \
                  order by Sales desc")

In [84]:
results.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [89]:
results=spark.sql("Select * from ( Select * from Software ) \
                  order by Sales desc").show()   # Using Derived Tables

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

